In [1]:
import pandas as pd
import openai
import time
import re

In [3]:
df = pd.read_excel("Antares Eazy Data .xlsx")
# df = pd.read_csv("real data lengkap.csv")
df

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Last Inbound Date,Assignee Agent Name,Assignee Agent Department,...,Resolved Agent Department,Conversation,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Weekend
0,32,113709,Whatsapp,-,~,2024-10-01 08:06:00,10/1/2024 8:25,10/1/2024 8:21,Budi Yulianto,Grup 2,...,Grup 2,"""customer : image""\n\ncustomer : mohon langkah...",antares eazy,antares eazy->gagal pairing camera general,Incident,"Pelanggan, Nita Setiawan, mengalami masalah de...",Terselesaikan,Sesuai,No,False
1,114,113258,Whatsapp,9/20/2024 17:51,9/20/2024 17:53,2024-10-01 10:47:00,10/1/2024 11:07,10/1/2024 10:51,Nur Awal Liyah,Grup 2,...,Grup 2,"""customer : siang""\n\nagent : selamat pagi bap...",antares eazy,eazy->gagal melihat rekaman,Incident,Pelanggan mengeluhkan bahwa CCTV tidak dapat m...,Belum Terselesaikan,Tidak Sesuai,No,False
2,194,113226,Whatsapp,9/30/2024 16:22,9/30/2024 17:03,2024-10-01 13:50:00,10/1/2024 14:19,10/1/2024 13:50,Nimatul HA,Grup 2,...,Grup 2,"""customer : selamat siang, mohon dibantu apaka...",antares eazy,antares eazy->tanya nomor resi,Request,"Pelanggan, Ibu Syifa, menanyakan informasi men...",Belum Terselesaikan,Tidak Sesuai,No,False
3,219,113772,Whatsapp,9/27/2024 22:00,9/27/2024 22:22,2024-10-01 14:48:00,10/1/2024 15:43,10/1/2024 15:34,Dinda Sukma Wardani,Grup 2,...,Grup 2,"""customer : lak""\n\ncustomer : kak""\n\ncustome...",antares eazy,antares eazy->informasi produk,Complaint,"Pelanggan, Bapak Didi, mengalami kebingungan t...",Sebagian Terselesaikan,Tidak Sesuai,No,False
4,230,113785,Whatsapp,10/1/2024 8:29,10/1/2024 8:29,2024-10-01 15:16:00,10/1/2024 15:50,10/1/2024 15:30,Nimatul HA,Grup 2,...,Grup 2,"""customer : halo kak""\n\ncustomer : kamera sud...",antares eazy,t-money->user tidak merespon,Request,"Pelanggan, Bapak Ricky, menghubungi layanan pe...",Sebagian Terselesaikan,Tidak Sesuai,No,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,26374,113448,Whatsapp,12/16/2024 9:47,12/16/2024 10:28,2024-12-16 11:15:00,12/16/2024 11:35,12/16/2024 11:21,Nur Awal Liyah,Grup 2,...,Grup 2,"""Customer : Untuk cloud memiliki penyimpanan u...",antares eazy,myindibiz->lainnya,Information,NaN,Terselesaikan,Tidak Sesuai,Yes,False
1352,28819,113347,Whatsapp,12/20/2024 16:25,12/20/2024 16:59,2024-12-20 17:10:00,12/20/2024 17:24,12/20/2024 17:10,Muhamad Fikri,Grup 2,...,Grup 2,"""Customer : Sudah bagus bosku""\n\nAgent : Sela...",antares eazy,t-money->user tidak merespon,Anomaly,NaN,Belum Terselesaikan,Sesuai,Yes,True
1353,29126,114216,Whatsapp,-,~,2024-12-21 19:20:00,12/21/2024 19:28,12/21/2024 19:27,Bambang Nur Abdillah,Grup 2,...,Grup 2,"""Customer : Kamera ini free atau gimana kak?""\...",antares eazy,myindibiz->informasi product,Information,NaN,Terselesaikan,Sesuai,No,True
1354,29730,114004,Whatsapp,12/22/2024 16:26,12/22/2024 16:35,2024-12-23 15:34:00,12/23/2024 16:39,12/23/2024 16:11,Anggita Windiana Sari,Grup 2,...,Grup 2,"""Customer : Paket sdh dtng""\n\nAgent : Selamat...",antares eazy,myindibiz->gagal login,Incident,NaN,Sebagian Terselesaikan,Sesuai,No,False


In [4]:
# df = df.iloc[5001:9194].reset_index(drop=True)
# df

In [5]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key='1FXoc1KNL0DTh7Zsnrum1BB21fnTz20QqcZT19NxA7iwRweubTMzJQQJ99AKACYeBjFXJ3w3AAABACOGniFk',
    api_version="2024-02-01",
    azure_endpoint = "https://openaifmseus.openai.azure.com/")

# deployment_name='gpt-35-turbo'
deployment_name='gpt-4o'

In [6]:
def clean_output(pertanyaan, jawaban):
    # Hapus strip dari pertanyaan
    pertanyaan = pertanyaan.replace("-", "").strip()
    
    # Pisahkan jawaban menjadi paragraf-paragraf
    jawaban_paragraf = jawaban.split("\n")
    
    # Ambil hanya paragraf pertama
    jawaban_cleaned = []
    for line in jawaban_paragraf:
        if line.strip() == "":
            break
        jawaban_cleaned.append(line.strip())
    
    jawaban_cleaned = " ".join(jawaban_cleaned)
    
    return pertanyaan, jawaban_cleaned

def extract_customer_agent(text):
    prompt = f"""
    Anda adalah ahli bahasa yang mengevaluasi percakapan layanan pelanggan. 
    Analisis percakapan dan ekstrak pertanyaan dan jawaban dari percakapan berikut '{text}' yang:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak memerlukan data real-time, status pesanan, tiket, atau akses ke sistem internal
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV)
    
    Berikan hasil dalam format berikut:
    - **Pertanyaan**: [Tuliskan pertanyaan dari customer yang sudah dirapikan dalam bentuk kalimat tanya yang jelas]
    - **Jawaban**: [Tuliskan jawaban dari agen yang sudah dirapikan menjadi kalimat yang SINGKAT, padat, jelas, dan mudah dipahami tanpa menyebut nama customer]

    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas, berikan "No data".
    Jika terdapat nama orang pada jawaban, hapus nama dan ganti/sesuaikan dengan kalimatnya saja.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.

    """
    try:
        chat_log = [{'role': 'user', 'content': prompt}]
        response = client.chat.completions.create(model=deployment_name, messages=chat_log)
        output = response.choices[0].message.content.strip()
        print(f"Output: {output}")

        pertanyaan, jawaban = "No data", "No data"
        if "**Pertanyaan**:" in output and "**Jawaban**:" in output:
            parts = output.split("**Jawaban**:")
            pertanyaan = parts[0].replace("**Pertanyaan**:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        
        # Cleaning the output
        pertanyaan, jawaban = clean_output(pertanyaan, jawaban)

        return pertanyaan, jawaban
    except Exception as e:
        print(f"Error: {e}")
        return "Error", "Error"

def analyze_conversations(df, batch_size=500, sleep_time=60):
    total_rows = len(df)
    
    # Menambahkan kolom baru untuk hasil ekstraksi
    if 'Pertanyaan' not in df.columns:
        df['Pertanyaan'] = ""
    if 'Jawaban' not in df.columns:
        df['Jawaban'] = ""

    for i in range(total_rows):
        try:
            text = df.iloc[i]['Conversation']
            
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue
            
            # Ekstraksi pertanyaan customer dan jawaban agen
            pertanyaan, jawaban = extract_customer_agent(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban
            
        except Exception as e:
            print(f"[{i}]> Error: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"
        
        # Batch processing untuk menghindari batasan API
        if (i + 1) % batch_size == 0:
            print(f"Processed {i + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    return df


In [7]:
%%time
analyze_conversations(df)
df

Output: - **Pertanyaan**: Bagaimana cara pairing kamera jika mengalami kegagalan koneksi?
- **Jawaban**: Anda dapat mengikuti panduan berikut: 
  1. Pastikan koneksi internet stabil.
  2. Reset kamera, tunggu sampai siap untuk pairing.
  3. Lampu indikator harus blink cepat (untuk merek selain eazycam).
  4. Gunakan WiFi 2.4GHz.
  5. Buka aplikasi Antares Eazy, pilih tambah perangkat, dan pilih merek serta jenis perangkat.
  6. Reset kamera dan konfirmasi langkah selanjutnya.
  7. Izinkan akses lokasi dan pastikan WiFi terhubung dengan HP.
  8. Arahkan QR code di aplikasi ke kamera hingga terdengar suara.
  9. Tunggu proses pairing selesai. Nama kamera bisa disesuaikan.
Output: No data.
Output: No data.
Output: - **Pertanyaan**: Apakah saya mendapatkan kamera ini sebagai bonus dari pemasangan wifi?
  - **Jawaban**: Tidak, berlangganan Antares Eazy dikenakan biaya per bulan dan bukan bonus dari pemasangan wifi.

- **Pertanyaan**: Apakah kami hanya berlangganan wifi?
  - **Jawaban**: Unt

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Last Inbound Date,Assignee Agent Name,Assignee Agent Department,...,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Weekend,Pertanyaan,Jawaban
0,32,113709,Whatsapp,-,~,2024-10-01 08:06:00,10/1/2024 8:25,10/1/2024 8:21,Budi Yulianto,Grup 2,...,antares eazy,antares eazy->gagal pairing camera general,Incident,"Pelanggan, Nita Setiawan, mengalami masalah de...",Terselesaikan,Sesuai,No,False,Bagaimana cara pairing kamera jika mengalami k...,Anda dapat mengikuti panduan berikut: 1. Pasti...
1,114,113258,Whatsapp,9/20/2024 17:51,9/20/2024 17:53,2024-10-01 10:47:00,10/1/2024 11:07,10/1/2024 10:51,Nur Awal Liyah,Grup 2,...,antares eazy,eazy->gagal melihat rekaman,Incident,Pelanggan mengeluhkan bahwa CCTV tidak dapat m...,Belum Terselesaikan,Tidak Sesuai,No,False,No data,No data
2,194,113226,Whatsapp,9/30/2024 16:22,9/30/2024 17:03,2024-10-01 13:50:00,10/1/2024 14:19,10/1/2024 13:50,Nimatul HA,Grup 2,...,antares eazy,antares eazy->tanya nomor resi,Request,"Pelanggan, Ibu Syifa, menanyakan informasi men...",Belum Terselesaikan,Tidak Sesuai,No,False,No data,No data
3,219,113772,Whatsapp,9/27/2024 22:00,9/27/2024 22:22,2024-10-01 14:48:00,10/1/2024 15:43,10/1/2024 15:34,Dinda Sukma Wardani,Grup 2,...,antares eazy,antares eazy->informasi produk,Complaint,"Pelanggan, Bapak Didi, mengalami kebingungan t...",Sebagian Terselesaikan,Tidak Sesuai,No,False,Apakah saya mendapatkan kamera ini sebagai bon...,"Tidak, berlangganan Antares Eazy dikenakan bia..."
4,230,113785,Whatsapp,10/1/2024 8:29,10/1/2024 8:29,2024-10-01 15:16:00,10/1/2024 15:50,10/1/2024 15:30,Nimatul HA,Grup 2,...,antares eazy,t-money->user tidak merespon,Request,"Pelanggan, Bapak Ricky, menghubungi layanan pe...",Sebagian Terselesaikan,Tidak Sesuai,No,False,Berapa kapasitas maksimal memori kamera?,Maksimal 128GB.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,26374,113448,Whatsapp,12/16/2024 9:47,12/16/2024 10:28,2024-12-16 11:15:00,12/16/2024 11:35,12/16/2024 11:21,Nur Awal Liyah,Grup 2,...,antares eazy,myindibiz->lainnya,Information,NaN,Terselesaikan,Tidak Sesuai,Yes,False,Apakah cloud memiliki penyimpanan unlimited un...,Video dapat diakses terus menerus jika kamera ...
1352,28819,113347,Whatsapp,12/20/2024 16:25,12/20/2024 16:59,2024-12-20 17:10:00,12/20/2024 17:24,12/20/2024 17:10,Muhamad Fikri,Grup 2,...,antares eazy,t-money->user tidak merespon,Anomaly,NaN,Belum Terselesaikan,Sesuai,Yes,True,No data,No data
1353,29126,114216,Whatsapp,-,~,2024-12-21 19:20:00,12/21/2024 19:28,12/21/2024 19:27,Bambang Nur Abdillah,Grup 2,...,antares eazy,myindibiz->informasi product,Information,NaN,Terselesaikan,Sesuai,No,True,Apakah kamera ini bisa didapatkan secara grati...,Kamera akan diterima sebagai paket bundling de...
1354,29730,114004,Whatsapp,12/22/2024 16:26,12/22/2024 16:35,2024-12-23 15:34:00,12/23/2024 16:39,12/23/2024 16:11,Anggita Windiana Sari,Grup 2,...,antares eazy,myindibiz->gagal login,Incident,NaN,Sebagian Terselesaikan,Sesuai,No,False,Apakah untuk pemantauan bisa digunakan lebih d...,Pemantauan bisa dilakukan dengan lebih dari sa...


In [6]:
df.to_csv("test_qna_myindibiz.csv", index=False) 

In [9]:
def clean_output(pertanyaan, jawaban):
    pertanyaan = pertanyaan.replace("-", "").strip()
    jawaban_paragraf = jawaban.split("\n")
    jawaban_cleaned = []
    for line in jawaban_paragraf:
        if line.strip() == "":
            break
        jawaban_cleaned.append(line.strip())
    jawaban_cleaned = " ".join(jawaban_cleaned)
    return pertanyaan, jawaban_cleaned

def extract_customer_agent(text):
    prompt = f"""
    Anda adalah ahli bahasa yang mengevaluasi percakapan layanan pelanggan. 
    Analisis percakapan dan ekstrak pertanyaan dan jawaban dari percakapan berikut '{text}' yang:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak memerlukan data real-time, status pesanan, tiket, atau akses ke sistem internal
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV)
    
    Berikan hasil dalam format berikut:
    - **Pertanyaan**: [Tuliskan pertanyaan dari customer yang sudah dirapikan dalam bentuk kalimat tanya yang jelas]
    - **Jawaban**: [Tuliskan jawaban dari agen yang sudah dirapikan menjadi kalimat yang SINGKAT, padat, jelas, dan mudah dipahami tanpa menyebut nama customer]

    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas, berikan "No data".
    Jika terdapat nama orang pada jawaban, hapus nama dan ganti/sesuaikan dengan kalimatnya saja.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.

    """
    try:
        chat_log = [{'role': 'user', 'content': prompt}]
        response = client.chat.completions.create(model=deployment_name, messages=chat_log)
        output = response.choices[0].message.content.strip()

        pertanyaan, jawaban = "No data", "No data"
        if "**Pertanyaan**:" in output and "**Jawaban**:" in output:
            parts = output.split("**Jawaban**:")
            pertanyaan = parts[0].replace("**Pertanyaan**:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        
        pertanyaan, jawaban = clean_output(pertanyaan, jawaban)

        return pertanyaan, jawaban
    except Exception as e:
        print(f"Error: {e}")
        return "Error", "Error"

def reprocess_errors(df):
    for i in df.index:
        if df.loc[i, 'Pertanyaan'] == "Error" or df.loc[i, 'Jawaban'] == "Error":
            try:
                text = df.loc[i, 'Conversation']
                if pd.isna(text) or text.strip() == "":
                    df.loc[i, 'Pertanyaan'] = "No data"
                    df.loc[i, 'Jawaban'] = "No data"
                    continue
                
                pertanyaan, jawaban = extract_customer_agent(text)
                df.loc[i, 'Pertanyaan'] = pertanyaan
                df.loc[i, 'Jawaban'] = jawaban

            except Exception as e:
                print(f"Reprocessing Error at row {i}: {e}")
                df.loc[i, 'Pertanyaan'] = "Error"
                df.loc[i, 'Jawaban'] = "Error"
    
    return df

In [10]:
%%time
df = reprocess_errors(df)
df

Error: 'NoneType' object has no attribute 'strip'
Error: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': True, 'detected': True}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}
CPU times: user 39.6 ms, sys: 15.8 ms, total: 55.4 ms
Wall time: 3.55 s


,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Last Inbound Date,Assignee Agent Name,Assignee Agent Department,...,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Weekend,Pertanyaan,Jawaban
0,32,113709,Whatsapp,-,~,2024-10-01 08:06:00,10/1/2024 8:25,10/1/2024 8:21,Budi Yulianto,Grup 2,...,antares eazy,antares eazy->gagal pairing camera general,Incident,"Pelanggan, Nita Setiawan, mengalami masalah de...",Terselesaikan,Sesuai,No,False,Bagaimana cara pairing kamera jika mengalami k...,Anda dapat mengikuti panduan berikut: 1. Pasti...
1,114,113258,Whatsapp,9/20/2024 17:51,9/20/2024 17:53,2024-10-01 10:47:00,10/1/2024 11:07,10/1/2024 10:51,Nur Awal Liyah,Grup 2,...,antares eazy,eazy->gagal melihat rekaman,Incident,Pelanggan mengeluhkan bahwa CCTV tidak dapat m...,Belum Terselesaikan,Tidak Sesuai,No,False,No data,No data
2,194,113226,Whatsapp,9/30/2024 16:22,9/30/2024 17:03,2024-10-01 13:50:00,10/1/2024 14:19,10/1/2024 13:50,Nimatul HA,Grup 2,...,antares eazy,antares eazy->tanya nomor resi,Request,"Pelanggan, Ibu Syifa, menanyakan informasi men...",Belum Terselesaikan,Tidak Sesuai,No,False,No data,No data
3,219,113772,Whatsapp,9/27/2024 22:00,9/27/2024 22:22,2024-10-01 14:48:00,10/1/2024 15:43,10/1/2024 15:34,Dinda Sukma Wardani,Grup 2,...,antares eazy,antares eazy->informasi produk,Complaint,"Pelanggan, Bapak Didi, mengalami kebingungan t...",Sebagian Terselesaikan,Tidak Sesuai,No,False,Apakah saya mendapatkan kamera ini sebagai bon...,"Tidak, berlangganan Antares Eazy dikenakan bia..."
4,230,113785,Whatsapp,10/1/2024 8:29,10/1/2024 8:29,2024-10-01 15:16:00,10/1/2024 15:50,10/1/2024 15:30,Nimatul HA,Grup 2,...,antares eazy,t-money->user tidak merespon,Request,"Pelanggan, Bapak Ricky, menghubungi layanan pe...",Sebagian Terselesaikan,Tidak Sesuai,No,False,Berapa kapasitas maksimal memori kamera?,Maksimal 128GB.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,26374,113448,Whatsapp,12/16/2024 9:47,12/16/2024 10:28,2024-12-16 11:15:00,12/16/2024 11:35,12/16/2024 11:21,Nur Awal Liyah,Grup 2,...,antares eazy,myindibiz->lainnya,Information,NaN,Terselesaikan,Tidak Sesuai,Yes,False,Apakah cloud memiliki penyimpanan unlimited un...,Video dapat diakses terus menerus jika kamera ...
1352,28819,113347,Whatsapp,12/20/2024 16:25,12/20/2024 16:59,2024-12-20 17:10:00,12/20/2024 17:24,12/20/2024 17:10,Muhamad Fikri,Grup 2,...,antares eazy,t-money->user tidak merespon,Anomaly,NaN,Belum Terselesaikan,Sesuai,Yes,True,No data,No data
1353,29126,114216,Whatsapp,-,~,2024-12-21 19:20:00,12/21/2024 19:28,12/21/2024 19:27,Bambang Nur Abdillah,Grup 2,...,antares eazy,myindibiz->informasi product,Information,NaN,Terselesaikan,Sesuai,No,True,Apakah kamera ini bisa didapatkan secara grati...,Kamera akan diterima sebagai paket bundling de...
1354,29730,114004,Whatsapp,12/22/2024 16:26,12/22/2024 16:35,2024-12-23 15:34:00,12/23/2024 16:39,12/23/2024 16:11,Anggita Windiana Sari,Grup 2,...,antares eazy,myindibiz->gagal login,Incident,NaN,Sebagian Terselesaikan,Sesuai,No,False,Apakah untuk pemantauan bisa digunakan lebih d...,Pemantauan bisa dilakukan dengan lebih dari sa...


In [11]:
# Filter the DataFrame for rows where 'Pertanyaan' or 'Jawaban' is "Error"
error_rows = df[(df['Pertanyaan'] == "Error") | (df['Jawaban'] == "Error")]
    
    # Display the filtered DataFrame
error_rows

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Last Inbound Date,Assignee Agent Name,Assignee Agent Department,...,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Weekend,Pertanyaan,Jawaban
433,8383,113554,Whatsapp,10/25/2024 19:12,10/25/2024 19:59,2024-10-26 13:43:00,10/26/2024 14:03,10/26/2024 13:45,Budi Yulianto,Grup 2,...,antares eazy,eazy->lainnya/others,Complaint,Pelanggan menghadapi masalah karena nomor 1500...,Sebagian Terselesaikan,Tidak Sesuai,Yes,True,Error,Error
1044,24297,113455,Email,-,~,2024-12-10 20:11:00,12/10/2024 20:12,12/10/2024 20:11,Sesep Irawan,Grup 2,...,antares eazy,eazy->lainnya/others,Request,"Pelanggan, yang mewakili Nestlé Mexico, menghu...",Belum Terselesaikan,Error: Unexpected Exception,No,False,Error,Error


In [25]:
df.to_csv('updated_gpt_qnatigabulanpls.csv', index=False)

In [12]:
def clean_text_pertanyaan(text):
    # Pisahkan teks menjadi paragraf berdasarkan newline ganda dan ambil paragraf terakhir
    paragraphs = text.strip().split('\n\n')
    return paragraphs[-1].strip()

# Terapkan fungsi untuk mendapatkan paragraf terakhir ke kolom 'Pertanyaan'
df['cleaned_pertanyaan'] = df['Pertanyaan'].apply(clean_text_pertanyaan)

In [13]:
df

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Last Inbound Date,Assignee Agent Name,Assignee Agent Department,...,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Weekend,Pertanyaan,Jawaban,cleaned_pertanyaan
0,32,113709,Whatsapp,-,~,2024-10-01 08:06:00,10/1/2024 8:25,10/1/2024 8:21,Budi Yulianto,Grup 2,...,antares eazy->gagal pairing camera general,Incident,"Pelanggan, Nita Setiawan, mengalami masalah de...",Terselesaikan,Sesuai,No,False,Bagaimana cara pairing kamera jika mengalami k...,Anda dapat mengikuti panduan berikut: 1. Pasti...,Bagaimana cara pairing kamera jika mengalami k...
1,114,113258,Whatsapp,9/20/2024 17:51,9/20/2024 17:53,2024-10-01 10:47:00,10/1/2024 11:07,10/1/2024 10:51,Nur Awal Liyah,Grup 2,...,eazy->gagal melihat rekaman,Incident,Pelanggan mengeluhkan bahwa CCTV tidak dapat m...,Belum Terselesaikan,Tidak Sesuai,No,False,No data,No data,No data
2,194,113226,Whatsapp,9/30/2024 16:22,9/30/2024 17:03,2024-10-01 13:50:00,10/1/2024 14:19,10/1/2024 13:50,Nimatul HA,Grup 2,...,antares eazy->tanya nomor resi,Request,"Pelanggan, Ibu Syifa, menanyakan informasi men...",Belum Terselesaikan,Tidak Sesuai,No,False,No data,No data,No data
3,219,113772,Whatsapp,9/27/2024 22:00,9/27/2024 22:22,2024-10-01 14:48:00,10/1/2024 15:43,10/1/2024 15:34,Dinda Sukma Wardani,Grup 2,...,antares eazy->informasi produk,Complaint,"Pelanggan, Bapak Didi, mengalami kebingungan t...",Sebagian Terselesaikan,Tidak Sesuai,No,False,Apakah saya mendapatkan kamera ini sebagai bon...,"Tidak, berlangganan Antares Eazy dikenakan bia...",Apakah saya mendapatkan kamera ini sebagai bon...
4,230,113785,Whatsapp,10/1/2024 8:29,10/1/2024 8:29,2024-10-01 15:16:00,10/1/2024 15:50,10/1/2024 15:30,Nimatul HA,Grup 2,...,t-money->user tidak merespon,Request,"Pelanggan, Bapak Ricky, menghubungi layanan pe...",Sebagian Terselesaikan,Tidak Sesuai,No,False,Berapa kapasitas maksimal memori kamera?,Maksimal 128GB.,Berapa kapasitas maksimal memori kamera?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,26374,113448,Whatsapp,12/16/2024 9:47,12/16/2024 10:28,2024-12-16 11:15:00,12/16/2024 11:35,12/16/2024 11:21,Nur Awal Liyah,Grup 2,...,myindibiz->lainnya,Information,NaN,Terselesaikan,Tidak Sesuai,Yes,False,Apakah cloud memiliki penyimpanan unlimited un...,Video dapat diakses terus menerus jika kamera ...,Apakah cloud memiliki penyimpanan unlimited un...
1352,28819,113347,Whatsapp,12/20/2024 16:25,12/20/2024 16:59,2024-12-20 17:10:00,12/20/2024 17:24,12/20/2024 17:10,Muhamad Fikri,Grup 2,...,t-money->user tidak merespon,Anomaly,NaN,Belum Terselesaikan,Sesuai,Yes,True,No data,No data,No data
1353,29126,114216,Whatsapp,-,~,2024-12-21 19:20:00,12/21/2024 19:28,12/21/2024 19:27,Bambang Nur Abdillah,Grup 2,...,myindibiz->informasi product,Information,NaN,Terselesaikan,Sesuai,No,True,Apakah kamera ini bisa didapatkan secara grati...,Kamera akan diterima sebagai paket bundling de...,Apakah kamera ini bisa didapatkan secara grati...
1354,29730,114004,Whatsapp,12/22/2024 16:26,12/22/2024 16:35,2024-12-23 15:34:00,12/23/2024 16:39,12/23/2024 16:11,Anggita Windiana Sari,Grup 2,...,myindibiz->gagal login,Incident,NaN,Sebagian Terselesaikan,Sesuai,No,False,Apakah untuk pemantauan bisa digunakan lebih d...,Pemantauan bisa dilakukan dengan lebih dari sa...,Apakah untuk pemantauan bisa digunakan lebih d...


In [14]:
df.to_csv('updated_qna_mytensupdate.csv', index=False)

In [14]:
# Menghapus baris yang berisi "No Data"
df = df[(df['cleaned_pertanyaan'] != 'No data') & (df['Jawaban'] != 'No data')]

# Reset indeks
df = df.reset_index(drop=True)

# Menampilkan dataframe yang telah dibersihkan
df

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Last Inbound Date,Assignee Agent Name,Assignee Agent Department,...,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Weekend,Pertanyaan,Jawaban,cleaned_pertanyaan
0,32,113709,Whatsapp,-,~,2024-10-01 08:06:00,10/1/2024 8:25,10/1/2024 8:21,Budi Yulianto,Grup 2,...,antares eazy->gagal pairing camera general,Incident,"Pelanggan, Nita Setiawan, mengalami masalah de...",Terselesaikan,Sesuai,No,False,Bagaimana cara pairing kamera jika mengalami k...,Anda dapat mengikuti panduan berikut: 1. Pasti...,Bagaimana cara pairing kamera jika mengalami k...
1,219,113772,Whatsapp,9/27/2024 22:00,9/27/2024 22:22,2024-10-01 14:48:00,10/1/2024 15:43,10/1/2024 15:34,Dinda Sukma Wardani,Grup 2,...,antares eazy->informasi produk,Complaint,"Pelanggan, Bapak Didi, mengalami kebingungan t...",Sebagian Terselesaikan,Tidak Sesuai,No,False,Apakah saya mendapatkan kamera ini sebagai bon...,"Tidak, berlangganan Antares Eazy dikenakan bia...",Apakah saya mendapatkan kamera ini sebagai bon...
2,230,113785,Whatsapp,10/1/2024 8:29,10/1/2024 8:29,2024-10-01 15:16:00,10/1/2024 15:50,10/1/2024 15:30,Nimatul HA,Grup 2,...,t-money->user tidak merespon,Request,"Pelanggan, Bapak Ricky, menghubungi layanan pe...",Sebagian Terselesaikan,Tidak Sesuai,No,False,Berapa kapasitas maksimal memori kamera?,Maksimal 128GB.,Berapa kapasitas maksimal memori kamera?
3,292,113240,Whatsapp,9/25/2024 18:02,9/25/2024 18:39,2024-10-01 18:13:00,10/1/2024 18:30,10/1/2024 18:22,Ikhsan Maulana,Grup 2,...,antares eazy->lainnya,Complaint,"Pelanggan, Bapak Ari, menghubungi layanan pela...",Sebagian Terselesaikan,Sesuai,No,False,Apakah bisa teknisi memasangkan CCTV?,"Mohon maaf, teknisi tidak bisa memasangkan CCT...",Apakah bisa teknisi memasangkan CCTV?
4,301,113255,Whatsapp,9/28/2024 19:47,9/28/2024 20:11,2024-10-01 19:20:00,10/1/2024 20:28,10/1/2024 20:27,Nofvel Maulana,Grup 2,...,eazy->gagal login antares eazy,Incident,"Pelanggan, Mustamin, menghadapi masalah dengan...",Terselesaikan,Sesuai,No,False,Bagaimana cara melakukan pairing atau aktivasi...,"Pastikan internet stabil, reset kamera, dan hu...",Bagaimana cara melakukan pairing atau aktivasi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,22829,113939,Whatsapp,12/5/2024 16:22,12/5/2024 16:50,2024-12-06 13:13:00,12/6/2024 13:51,12/6/2024 13:50,Syawal Basri P,Grup 1,...,myindibiz->informasi hsi,Request,NaN,Terselesaikan,Sesuai,No,False,Apa perbedaan antara Paket Promo Hari Guru Nas...,Perbedaannya terletak pada rasio kecepatan. HS...,Apa perbedaan antara Paket Promo Hari Guru Nas...
591,26278,113453,Whatsapp,12/13/2024 8:56,12/13/2024 10:19,2024-12-16 09:47:00,12/16/2024 10:28,12/16/2024 10:25,Nur Awal Liyah,Grup 2,...,myindibiz->gangguan kendala teknisi,Information,NaN,Sebagian Terselesaikan,Sesuai,No,False,"Bagaimana dengan teknisi CCTV, karena konfirma...","Pemasangan kamera dilakukan secara mandiri, ji...","Bagaimana dengan teknisi CCTV, karena konfirma..."
592,26374,113448,Whatsapp,12/16/2024 9:47,12/16/2024 10:28,2024-12-16 11:15:00,12/16/2024 11:35,12/16/2024 11:21,Nur Awal Liyah,Grup 2,...,myindibiz->lainnya,Information,NaN,Terselesaikan,Tidak Sesuai,Yes,False,Apakah cloud memiliki penyimpanan unlimited un...,Video dapat diakses terus menerus jika kamera ...,Apakah cloud memiliki penyimpanan unlimited un...
593,29126,114216,Whatsapp,-,~,2024-12-21 19:20:00,12/21/2024 19:28,12/21/2024 19:27,Bambang Nur Abdillah,Grup 2,...,myindibiz->informasi product,Information,NaN,Terselesaikan,Sesuai,No,True,Apakah kamera ini bisa didapatkan secara grati...,Kamera akan diterima sebagai paket bundling de...,Apakah kamera ini bisa didapatkan secara grati...


In [14]:
df.to_csv('antareseazy_baru.csv', index=False)